# Try Reproducing Everything to Stock

In [1]:
import sys
import os
sys.path.append(os.path.abspath("D:\\Python\\commodity_futures_price"))

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from model.savvy_sh import SavvySh

import yfinance as yf

from model.factor import Factor

### Download Necessary Data From yfinance (if haven't downloaded)

In [2]:
ticker = "AAPL"
# download if not found in ../../data/yfinance/
if not os.path.exists(f"../../data/yfinance/{ticker}.csv"):
    data = yf.Ticker(ticker)
    data = data.history(period="15Y", repair=True)
    data.index = data.index.date
    data.to_csv(f"../../data/yfinance/{ticker}.csv")
data = pd.read_csv(f"../../data/yfinance/{ticker}.csv", index_col=0, parse_dates=True)
# check if there is any row with Repair = True
data[data['Repaired?'] == True]

factor = Factor(data, ticker=ticker)

### Momentum

In [3]:
factor.add_momentum()

### Skewness

In [4]:
factor.add_skewness()

### Volatility

In [5]:
factor.add_Volatility()

### Open Interest

In [6]:
factor.add_Open_Interest()

### Value

In [7]:
factor.add_Value()

### Inflation Beta

In [15]:
factor.add_inflation_beta()

In [14]:
print(data.dropna())

                  Open        High         Low       Close     Volume  \
2015-10-28   26.277163   26.809764   26.081652   26.803020  342205600   
2015-10-29   26.674922   27.122127   26.578290   27.086170  204909200   
2015-10-30   27.189554   27.241242   26.843476   26.854713  197461200   
2015-11-02   27.146849   27.272695   26.879425   27.232244  128813200   
2015-11-03   27.144602   27.751360   27.124375   27.544613  182076000   
...                ...         ...         ...         ...        ...   
2025-06-24  202.360334  203.209377  199.973044  200.072937   54064000   
2025-06-25  201.221637  203.439121  200.392576  201.331512   39525700   
2025-06-26  201.201646  202.410281  199.233893  200.772141   50799100   
2025-06-27  201.661130  202.989624  199.773273  200.852051   73188600   
2025-06-30  201.780987  207.154893  199.034105  204.937408   91912800   

            Dividends  Stock Splits  Repaired?  MOM_vol_adj  skewness  \
2015-10-28        0.0           0.0      False    